Getting Started with Gym
==============

Gym is a toolkit for developing and comparing reinforcement learning algorithms that makes no assumptions about the structure of your agent.

The gym library is a collection of test problems — environments — that you can use to work out your reinforcement learning algorithms. These environments have a shared interface, allowing you to write general algorithms.

Environments
--------------

Here’s a bare minimum example of getting something running. This will run an instance of the CartPole-v0 environment for 1000 timesteps, rendering the environment at each step. You should see a window pop up rendering the classic cart-pole problem:

In [ ]:
import gym
env = gym.make('MsPacman-v0')
env.reset()
for _ in range(500):
    env.render()
    env.step(env.action_space.sample()) # take a random action
env.close()

If you’d like to see some other environments in action, try replacing MsPacman-v0 above with something like MountainCar-v0, or Hopper-v1 (requires the MuJoCo dependencies). Environments all descend from the Env base class.

Note that if you’re missing any dependencies, you should get a helpful error message telling you what you’re missing. Installing a missing dependency is generally pretty simple. You’ll also need a MuJoCo license for Hopper-v1.

Observations
--------------

If we ever want to do better than take random actions at each step, it’d probably be good to actually know what our actions are doing to the environment.

The environment’s step function returns exactly what we need. In fact, step returns four values. These are:

1. `observation` (object): an environment-specific object representing your observation of the environment. For example, pixel data from a camera, joint angles and joint velocities of a robot, or the board state in a board game.
2. `reward` (float): amount of reward achieved by the previous action. The scale varies between environments, but the goal is always to increase your total reward.
3. `done` (boolean): whether it’s time to reset the environment again. Most (but not all) tasks are divided up into well-defined episodes, and done being True indicates the episode has terminated. (For example, perhaps the pole tipped too far, or you lost your last life.)
4. `info` (dict): diagnostic information useful for debugging. It can sometimes be useful for learning (for example, it might contain the raw probabilities behind the environment’s last state change). However, official evaluations of your agent are not allowed to use this for learning.

This is just an implementation of the classic “agent-environment loop”. Each timestep, the agent chooses an action, and the environment returns an observation and a reward.

The process gets started by calling `reset()`, which returns an initial observation. So a more proper way of writing the previous code would be to respect the `done` flag:

In [ ]:
env = gym.make('MsPacman-v0')
for i_episode in range(2):
    observation = env.reset()
    for t in range(1000):
        env.render()
        action = env.action_space.sample()
        observation, reward, done, info = env.step(action)
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            break
env.close()

Spaces
--------

In the examples above, we’ve been sampling random actions from the environment’s action space. But what actually are those actions? Every environment comes with an `action_space` and an `observation_space`. These attributes are of type `Space`, and they describe the format of valid actions and observations:

In [ ]:
env = gym.make('MsPacman-v0')
print(env.action_space)
print(env.observation_space)

The `Discrete` space allows a fixed range of non-negative numbers, so in this case valid actions are either 0 to 8. The `Box` space represents an n-dimensional box, so valid observations will be an RGB image of size 210x160. We can also check the Box’s bounds:

In [ ]:
print(env.observation_space.high[:, :, 0])
print(env.observation_space.low[:, :, 0])

This introspection can be helpful to write generic code that works for many different environments. `Box` and `Discrete` are the most common `Spaces`. You can sample from a `Space` or check that something belongs to it:

In [ ]:
from gym import spaces
space = spaces.Discrete(8) # Set with 8 elements {0, 1, 2, ..., 7}
x = space.sample()
print(x)
print(space.contains(x))
print(space.n == 8)

Available Environments
----------------------------

Gym comes with a diverse suite of environments that range from easy to difficult and involve many different kinds of data. View the full list of environments to get the birds-eye view.

1. Classic control and toy text: complete small-scale tasks, mostly from the RL literature. They’re here to get you started.
2. Algorithmic: perform computations such as adding multi-digit numbers and reversing sequences. One might object that these tasks are easy for a computer. The challenge is to learn these algorithms purely from examples. These tasks have the nice property that it’s easy to vary the difficulty by varying the sequence length.
3. Atari: play classic Atari games. We’ve integrated the Arcade Learning Environment (which has had a big impact on reinforcement learning research) in an easy-to-install form.
4. 2D and 3D robots: control a robot in simulation. These tasks use the MuJoCo physics engine, which was designed for fast and accurate robot simulation.

Wrappers
---------

Wrappers are used to transform an environment in a modular way. For example, here is a wrapper to downsample the RGB observations of an environment and convert them to grayscale.

In [ ]:
import cv2
import numpy as np

class WarpFrame(gym.ObservationWrapper):
    def __init__(self, env):
        """Warp frames to 84x84 as done in the Nature paper and later work."""
        gym.ObservationWrapper.__init__(self, env)
        self.width = 84
        self.height = 84
        self.observation_space = spaces.Box(low=0, high=255,
            shape=(self.height, self.width, 1), dtype=np.uint8)

    def observation(self, frame):
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        frame = cv2.resize(frame, (self.width, self.height), interpolation=cv2.INTER_AREA)
        return frame[:, :, None]

We now wrap MsPacman-v0 using `WarpFrame`:

In [ ]:
env = gym.make('MsPacman-v0')
print(env.observation_space.shape)
env = WarpFrame(env)
print(env.observation_space.shape)

Exercise
---------

Complete the missing lines to create your own gym environment.

In [ ]:
import math
import gym
from gym import spaces, logger
from gym.utils import seeding
import numpy as np

class CartPoleEnv(gym.Env):
    metadata = {
        'render.modes': ['human', 'rgb_array'],
        'video.frames_per_second' : 50
    }

    def __init__(self):
        self.gravity = 9.8
        self.masscart = 1.0
        self.masspole = 0.1
        self.total_mass = (self.masspole + self.masscart)
        self.length = 0.5 # actually half the pole's length
        self.polemass_length = (self.masspole * self.length)
        self.force_mag = 10.0
        self.tau = 0.02  # seconds between state updates

        # Angle at which to fail the episode
        self.theta_threshold_radians = 12 * 2 * math.pi / 360
        self.x_threshold = 2.4

        # Angle limit set to 2 * theta_threshold_radians so failing observation is still within bounds
        high = np.array([
            self.x_threshold * 2,
            np.finfo(np.float32).max,
            self.theta_threshold_radians * 2,
            np.finfo(np.float32).max])
        low = -high

        # TODO: Define the action space. There are 2 possible actions: 
        # apply force from the left and apply force from the right
        self.action_space = ...
        
        # TODO: Define the observation space. The range of values is defined above using low and high.
        self.observation_space = ...

        self.viewer = None
        self.state = None

    def step(self, action):
        # TODO: check that the action is valid i.e. action is contained in the action_space
        
        state = self.state
        x, x_dot, theta, theta_dot = state
        
        # TODO: use action to decide force to apply to the cart (i.e. whether to use -force_mag or force_mag)
        force = ...
        
        costheta = math.cos(theta)
        sintheta = math.sin(theta)
        temp = (force + self.polemass_length * theta_dot * theta_dot * sintheta) / self.total_mass
        thetaacc = (self.gravity * sintheta - costheta* temp) / (self.length * (4.0/3.0 - self.masspole * costheta * costheta / self.total_mass))
        xacc  = temp - self.polemass_length * thetaacc * costheta / self.total_mass
        
        # Use the equations of motion to update the state variables
        x = ...
        x_dot = ...
        theta = ...
        theta_dot = ...
        
        self.state = (x,x_dot,theta,theta_dot)
        done =  x < -self.x_threshold \
                or x > self.x_threshold \
                or theta < -self.theta_threshold_radians \
                or theta > self.theta_threshold_radians
        done = bool(done)

        # TODO: if the pole hasn't fallen over the reward is 1 else set it to 0.
        reward = ...
        
        return np.array(self.state), reward, done, {}

    def reset(self):
        # TODO: Reset the environment by sampling the state uniformly in the range of -0.05 to -0.05 
        self.state = ...

        return np.array(self.state)

    def render(self, mode='human'):
        screen_width = 600
        screen_height = 400

        world_width = self.x_threshold*2
        scale = screen_width/world_width
        carty = 100 # TOP OF CART
        polewidth = 10.0
        polelen = scale * 1.0
        cartwidth = 50.0
        cartheight = 30.0

        if self.viewer is None:
            from gym.envs.classic_control import rendering
            self.viewer = rendering.Viewer(screen_width, screen_height)
            l,r,t,b = -cartwidth/2, cartwidth/2, cartheight/2, -cartheight/2
            axleoffset =cartheight/4.0
            cart = rendering.FilledPolygon([(l,b), (l,t), (r,t), (r,b)])
            self.carttrans = rendering.Transform()
            cart.add_attr(self.carttrans)
            self.viewer.add_geom(cart)
            l,r,t,b = -polewidth/2,polewidth/2,polelen-polewidth/2,-polewidth/2
            pole = rendering.FilledPolygon([(l,b), (l,t), (r,t), (r,b)])
            pole.set_color(.8,.6,.4)
            self.poletrans = rendering.Transform(translation=(0, axleoffset))
            pole.add_attr(self.poletrans)
            pole.add_attr(self.carttrans)
            self.viewer.add_geom(pole)
            self.axle = rendering.make_circle(polewidth/2)
            self.axle.add_attr(self.poletrans)
            self.axle.add_attr(self.carttrans)
            self.axle.set_color(.5,.5,.8)
            self.viewer.add_geom(self.axle)
            self.track = rendering.Line((0,carty), (screen_width,carty))
            self.track.set_color(0,0,0)
            self.viewer.add_geom(self.track)

        if self.state is None: return None

        x = self.state
        cartx = x[0]*scale+screen_width/2.0 # MIDDLE OF CART
        self.carttrans.set_translation(cartx, carty)
        self.poletrans.set_rotation(-x[2])

        return self.viewer.render(return_rgb_array = mode=='rgb_array')

    def close(self):
        if self.viewer: self.viewer.close()

In [ ]:
# Typically you'd register the environment and use gym.make but we're skipping this for convinience.
env = CartPoleEnv()
obs = env.reset()
for _ in range(20):
    print(obs)
    env.render()
    obs, reward, done, _ = env.step(env.action_space.sample()) # take a random action
    if done:
        break
env.close()